In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
import random

In [ ]:
user_url = 'https://raw.githubusercontent.com/qannisa/dummydataset/main/user_feature.csv'
scholarship_url = 'https://raw.githubusercontent.com/qannisa/dummydataset/main/scholarship_features_f.csv'

user_file = pd.read_csv(user_url)
scholarship_file = pd.read_csv(scholarship_url)



In [ ]:
user_file

,honors,experience,gpa,scholarship_tag,scholarship_name
0,0,2,3.77,10,Beasiswa Bakti BCA
1,0,3,3.95,10,Beasiswa Bakti BCA
2,0,1,3.46,10,Beasiswa Bakti BCA
3,0,2,3.95,10,Beasiswa Bakti BCA
4,3,3,3.57,10,Beasiswa Bakti BCA
...,...,...,...,...,...
11995,2,3,3.45,32,Beasiswa XL Future Leadership
11996,2,3,3.74,32,Beasiswa XL Future Leadership
11997,2,2,3.67,32,Beasiswa XL Future Leadership
11998,1,3,3.64,32,Beasiswa XL Future Leadership


In [ ]:
scholarship_file

,honors,experiences,gpa,scholarship_tag,scholarship_name
0,0,1,3.25,10,Beasiswa BCA
1,0,1,3.25,10,Beasiswa BCA
2,0,1,3.25,10,Beasiswa BCA
3,0,1,3.25,10,Beasiswa BCA
4,0,1,3.25,10,Beasiswa BCA
...,...,...,...,...,...
11995,0,1,2.80,32,Beasiswa XL Future Leader
11996,0,1,2.80,32,Beasiswa XL Future Leader
11997,0,1,2.80,32,Beasiswa XL Future Leader
11998,0,1,2.80,32,Beasiswa XL Future Leader


In [ ]:
num_user_features = user_file.shape[1] - 1         # remove userid, rating count and ave rating during training
num_item_features = scholarship_file.shape [1] - 1 # remove movie id at train time
print(num_user_features, num_item_features)

user_train = user_file.values.tolist()
item_train = scholarship_file.values.tolist()

random.shuffle(user_train)
random.shuffle(item_train)

print(user_train)
print(item_train)

y_train = [row[3] for row in user_train]
print(y_train)

user_train = np.array(user_train)
item_train = np.array(item_train)
y_train = np.array(y_train)

user_train_f = user_train[:, :-1]
item_train_f = item_train[:, :-1]
y_train_f = y_train.reshape(-1, 1)

print(user_train_f)
print(item_train_f)

print(user_train_f.shape)
print(item_train_f.shape)
print(y_train_f.shape)

4 4
[[2, 4, 3.72, 18, 'Beasiswa BSI'], [0, 3, 3.73, 18, 'Beasiswa BSI'], [0, 4, 3.65, 18, 'Beasiswa BSI'], [1, 4, 3.25, 16, 'Beasiswa BRIlian Future Scholarship'], [4, 2, 3.52, 20, 'Beasiswa CIMB Niaga'], [0, 4, 3.69, 20, 'Beasiswa CIMB Niaga'], [0, 4, 3.89, 24, 'Beasiswa Djarum'], [0, 3, 3.6, 26, 'Beasiswa Karya Salemba Empat'], [4, 4, 3.34, 12, 'Beasiswa Unggulan'], [4, 4, 3.19, 24, 'Beasiswa Djarum'], [0, 1, 3.61, 10, 'Beasiswa Bakti BCA'], [0, 3, 3.88, 30, 'Beasiswa Tanoto'], [2, 2, 3.07, 28, 'Beasiswa Pertamina'], [2, 1, 3.76, 22, 'Beasiswa DataPrint'], [4, 4, 3.87, 12, 'Beasiswa Unggulan'], [0, 2, 3.67, 10, 'Beasiswa Bakti BCA'], [2, 3, 3.4, 12, 'Beasiswa Unggulan'], [2, 4, 3.19, 12, 'Beasiswa Unggulan'], [0, 3, 3.69, 28, 'Beasiswa Pertamina'], [0, 3, 3.55, 28, 'Beasiswa Pertamina'], [0, 4, 3.24, 18, 'Beasiswa BSI'], [1, 4, 3.7, 18, 'Beasiswa BSI'], [0, 4, 3.24, 26, 'Beasiswa Karya Salemba Empat'], [0, 3, 3.06, 26, 'Beasiswa Karya Salemba Empat'], [2, 3, 3.2, 26, 'Beasiswa Karya 

In [ ]:
# scale training data
item_train_unscaled= item_train_f
user_train_unscaled = user_train_f
y_train_unscaled= y_train_f

scalerItem = StandardScaler()
scalerItem.fit(item_train_f)
item_train = scalerItem.transform(item_train_f)

scalerUser = StandardScaler()
scalerUser.fit(user_train_f)
user_train = scalerUser.transform(user_train_f)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train_f.reshape(-1, 1))
y_train = scalerTarget.transform(y_train_f.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

user_train_f = np.asarray(user_train_f).astype('float32')
item_train_f= np.asarray(item_train_f).astype('float32')
y_train_f= np.asarray(y_train_f).astype('float32')

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train_f)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train_f)))

False
False


In [ ]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
      tf.keras.layers.Dense(128, activation="relu", name= "layer1"),
      tf.keras.layers.Dense(64, activation="relu", name= "layer2"),
      tf.keras.layers.Dense(num_outputs, activation="linear", name="layer3")

    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
      tf.keras.layers.Dense(128, activation="relu", name= "layer1"),
      tf.keras.layers.Dense(64, activation="relu", name= "layer2"),
      tf.keras.layers.Dense(num_outputs, activation="linear", name="layer3")

    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 input_14 (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 sequential_14 (Sequential)  (None, 32)                   10976     ['input_13[0][0]']            
                                                                                                  
 sequential_15 (Sequential)  (None, 32)                   10976     ['input_14[0][0]']            
                                                                                            

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
print(user_train)
print(item_train)
print(y_train)

[[ 0.38788092  0.9557323   0.6905698  -0.4345241 ]
 [-1.0553969  -0.0321047   0.7255612  -0.4345241 ]
 [-1.0553969   0.9557323   0.4456303  -0.4345241 ]
 ...
 [ 1.8311586  -2.9956157  -0.11423229 -0.14484136]
 [-1.0553969  -2.007779    0.900518   -0.7242068 ]
 [ 0.38788092 -1.0199418   0.6205871  -1.593255  ]]
[[ 1.183216   -2.236068   -0.35300675 -1.3035723 ]
 [-0.8451542   0.44721362 -0.35300675 -1.0138896 ]
 [-0.8451542   0.44721362 -0.35300675  1.0138896 ]
 ...
 [-0.8451542   0.44721362  1.5724847  -1.593255  ]
 [ 1.183216    0.44721362 -0.35300675  0.14484136]
 [ 1.183216    0.44721362  1.5724847  -0.14484136]]
[[-0.27272725]
 [-0.27272725]
 [-0.27272725]
 ...
 [-0.09090912]
 [-0.4545455 ]
 [-1.        ]]


In [ ]:
tf.random.set_seed(1)

#user_train= np.array(user_train)
#item_train = np.array(item_train)
#y_train= np.array(y_train)

#user_train_f = user_train[:, :-1]
#item_train_f = item_train[:, :-1]
#y_train_f = y_train.reshape(-1, 1)

#user_train_tensor = np.asarray(user_train_f).astype('float32')
#item_train_tensor= np.asarray(item_train_f).astype('float32')
#y_train_tensor= np.asarray(y_train_f).astype('float32')

#print(user_train_tensor.dtype)
#print(item_train_tensor.dtype)
#print(y_train_tensor.dtype)


model.fit([user_train_f, item_train_f], y_train_f, epochs=30)


Epoch 1/30
375/375 [==============================] - 3s 4ms/step - loss: 448.2234
Epoch 2/30
375/375 [==============================] - 2s 5ms/step - loss: 447.6683
Epoch 3/30
375/375 [==============================] - 2s 5ms/step - loss: 447.6670
Epoch 4/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 5/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 6/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 7/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 8/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 9/30
375/375 [==============================] - 1s 3ms/step - loss: 447.6667
Epoch 10/30
189/375 [==============>...............] - ETA: 0s - loss: 451.1627

KeyboardInterrupt: ignored